In [1]:
from google.colab import files
import pandas as pd

# Subir archivos
uploaded = files.upload()

# Verás un selector para subir los archivos desde tu PC
# Luego los puedes cargar con pandas
covid_df = pd.read_csv("covid_19_clean_complete_CLEAN.csv")
usa_df = pd.read_csv("usa_county_wise_CLEAN.csv")
worldometer_df = pd.read_csv("worldometer_data_CLEAN.csv")

Saving covid_19_clean_complete_CLEAN.csv to covid_19_clean_complete_CLEAN.csv
Saving usa_county_wise_CLEAN.csv to usa_county_wise_CLEAN.csv
Saving worldometer_data_CLEAN.csv to worldometer_data_CLEAN.csv


# 1. Selección de datos relevantes

En este paso filtramos las columnas más importantes para el análisis.
Por ejemplo, en Worldometer Data no utilizaremos todas las métricas, solo aquellas relacionadas con casos, muertes y población.

In [2]:
# Selección de columnas relevantes en Worldometer
worldometer_df = worldometer_df[[
    "Country/Region", "Continent", "Population",
    "TotalCases", "TotalDeaths", "TotalRecovered",
    "ActiveCases", "TotalTests"
]]

worldometer_df.head()


,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,TotalTests
0,USA,North America,3.311981e+08,5032179,162804.0,2576668.0,2292707.0,63139605.0
1,Brazil,South America,2.127107e+08,2917562,98644.0,2047660.0,771258.0,13206188.0
2,India,Asia,1.381345e+09,2025409,41638.0,1377384.0,606387.0,22149351.0
3,Russia,Europe,1.459409e+08,871894,14606.0,676357.0,180931.0,29716907.0
4,South Africa,Africa,5.938157e+07,538184,9604.0,387316.0,141264.0,3149807.0


# 2. Limpieza de datos (missing values)

Ya realizamos limpieza en Kedro, pero aquí mostramos cómo se rellenaron valores nulos con "Unknown" o 0 según el tipo de variable.

In [3]:
# Verificación de nulos
print(worldometer_df.isnull().sum())

# Reemplazo de nulos en categóricas y numéricas (ejemplo)
worldometer_df["Continent"] = worldometer_df["Continent"].fillna("Unknown")
worldometer_df["TotalCases"] = worldometer_df["TotalCases"].fillna(0)


Country/Region    0
Continent         0
Population        0
TotalCases        0
TotalDeaths       0
TotalRecovered    0
ActiveCases       0
TotalTests        0
dtype: int64


# 3. Construcción de nuevas variables (Feature Engineering)

Creamos nuevas métricas para enriquecer el análisis:

In [4]:
# Casos por cada 100 mil habitantes
worldometer_df["Cases_per_100k"] = (worldometer_df["TotalCases"] / worldometer_df["Population"]) * 100000

# Muertes por cada 100 mil habitantes
worldometer_df["Deaths_per_100k"] = (worldometer_df["TotalDeaths"] / worldometer_df["Population"]) * 100000

# Tasa de letalidad (%)
worldometer_df["FatalityRate"] = (worldometer_df["TotalDeaths"] / worldometer_df["TotalCases"]) * 100


# 4. Integración de múltiples datasets

Podemos unir los datasets covid_19_clean_complete y worldometer_df por país para enriquecer la información.

In [5]:
# Integración básica por país
merged_df = covid_df.merge(
    worldometer_df,
    left_on="Country/Region",
    right_on="Country/Region",
    how="inner"
)

merged_df.head()


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,TotalTests,Cases_per_100k,Deaths_per_100k,FatalityRate
0,Unknown,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean,Asia,39009447.0,36896,1298.0,25840.0,9758.0,90396.0,94.582217,3.327399,3.517997
1,Unknown,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe,Europe,2877470.0,6016,188.0,3155.0,2673.0,38997.0,209.072553,6.533517,3.125000
2,Unknown,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa,Africa,43926079.0,33626,1273.0,23238.0,9115.0,0.0,76.551335,2.898051,3.785761
3,Unknown,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe,Europe,77278.0,944,52.0,828.0,64.0,3750.0,1221.563705,67.289526,5.508475
4,Unknown,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa,Africa,32956300.0,1483,64.0,520.0,899.0,64747.0,4.499898,0.194197,4.315577


# 5. Formateo y transformación

Normalizamos nombres de columnas y aseguramos tipos de datos correctos.

In [ ]:
# Normalización de nombres de columnas
merged_df.columns = merged_df.columns.str.replace(" ", "_").str.lower()

# Conversión de fechas
merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")
